# Pymaceuticals Inc.
---

### Analysis

- Add your analysis here.
 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
Complete_Study_df = pd.merge(study_results, mouse_metadata, how="outer", on = "Mouse ID")
# Display the data table for preview
print(len(Complete_Study_df["Mouse ID"]))
Complete_Study_df.head()

1893


,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [2]:
# Checking the number of mice.
len(Complete_Study_df["Mouse ID"].unique())

249

In [10]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
Dup_ID = Complete_Study_df["Mouse ID"].loc[Complete_Study_df.duplicated(subset=["Mouse ID","Timepoint"])].unique()
Dup_ID

array(['g989'], dtype=object)

In [11]:
# Optional: Get all the data for the duplicate mouse ID. 
#Drop_ID = Complete_Study_df.loc[Complete_Study_df.duplicated(subset=["Mouse ID","Timepoint"],keep=False)]
#Drop_ID
Drop_ID = Complete_Study_df.loc[Complete_Study_df["Mouse ID"]==Dup_ID[0]]

In [12]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.

#Merge original complete study dataframe with duplicate IDs and add an indicator column
Complete_Study_df = Complete_Study_df.merge(Drop_ID, how='left', indicator=True)

#Perform an operation that only keeps rows that were not identified in "Drop_ID"
Complete_Study_df = Complete_Study_df[Complete_Study_df['_merge']=='left_only']

#Drop the indicator column
Complete_Study_df = Complete_Study_df.drop(columns = "_merge")

In [14]:
# Checking the number of mice in the clean DataFrame.
len(Complete_Study_df["Mouse ID"].unique())

248

## Summary Statistics

In [19]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
Average = Complete_Study_df["Tumor Volume (mm3)"].mean()
print(Average)
Median = Complete_Study_df["Tumor Volume (mm3)"].median()
print(Median)
Variance = Complete_Study_df["Tumor Volume (mm3)"].var()
print(Variance)
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.



50.435293232255326
48.933453655
79.46290714557342


In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
# and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line.


## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using Pandas.



In [ ]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using pyplot.



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.



## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin



In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen



## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen

